In [2]:
%load_ext autoreload
%autoreload 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [15]:
import ControlBurn as cb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import LogisticRegression as lr

In [30]:
df_rain = pd.read_csv('datasets/weatherAUS_clean.csv')
rain_feats = df_rain.columns.values
rain_x = df_rain[rain_feats[1:-1]]
rain_y = df_rain[rain_feats[-1]]
rain_x_tr, rain_x_te, rain_y_tr, rain_y_te = train_test_split(rain_x, rain_y, test_size=0.2)

In [31]:
rain_x.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindGustDir_E,WindGustDir_W,WindDir9am_N,WindDir9am_S,WindDir9am_E,WindDir9am_W,WindDir3pm_N,WindDir3pm_S,WindDir3pm_E,WindDir3pm_W
0,13.4,22.9,0.6,24.666201,7.20069,44.0,20.0,24.0,71.0,22.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
1,7.4,25.1,0.0,24.666201,7.20069,44.0,4.0,22.0,44.0,25.0,...,0.0,2.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
2,12.9,25.7,0.0,24.666201,7.20069,46.0,19.0,26.0,38.0,30.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
3,9.2,28.0,0.0,24.666201,7.20069,24.0,11.0,9.0,45.0,16.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,17.5,32.3,1.0,24.666201,7.20069,41.0,7.0,20.0,82.0,33.0,...,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0


In [32]:
rain_feats

array(['Unnamed: 0', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm',
       'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday',
       'WindGustDir_N', 'WindGustDir_S', 'WindGustDir_E', 'WindGustDir_W',
       'WindDir9am_N', 'WindDir9am_S', 'WindDir9am_E', 'WindDir9am_W',
       'WindDir3pm_N', 'WindDir3pm_S', 'WindDir3pm_E', 'WindDir3pm_W',
       'RainTomorrow'], dtype=object)

In [33]:
rain_y_te_true = rain_y_te[rain_y_te == 1]
rain_x_te_true = rain_x_te[rain_y_te == 1]
rain_y_te_false = rain_y_te[rain_y_te == 0]
rain_x_te_false = rain_x_te[rain_y_te == 0]

In [34]:
U, S, V = np.linalg.svd(rain_x_tr[:5000])

In [35]:
S 
# demonstrate features selected by PCA would do poorly in RF but do well with lasso
# normalize every feature and try again

array([1.01890641e+05, 1.89226240e+03, 1.16673248e+03, 8.28715900e+02,
       7.34089652e+02, 7.01114584e+02, 5.54174230e+02, 4.33299035e+02,
       3.99090822e+02, 2.17542155e+02, 1.72791760e+02, 1.18711755e+02,
       1.01616516e+02, 9.62688959e+01, 8.50055700e+01, 8.21804107e+01,
       6.37943142e+01, 5.13001735e+01, 4.98827029e+01, 4.59300223e+01,
       3.93164197e+01, 3.70277834e+01, 3.65528633e+01, 3.10410384e+01,
       2.91991280e+01, 2.86994364e+01, 2.69736283e+01, 2.33651891e+01,
       2.13066395e+01])

In [36]:
clf = lr(penalty='l2').fit(rain_x_tr, rain_y_tr)
print(clf.score(rain_x_te, rain_y_te), clf.score(rain_x_te_true, rain_y_te_true), clf.score(rain_x_te_false, rain_y_te_false))
print('Coefficients:', clf.coef_)

0.8457590935216356 0.48176263654878726 0.9494645777285435
Coefficients: [[-0.01216841  0.01630153  0.02089404 -0.00416344 -0.13223034  0.06300355
  -0.01579281 -0.02827878  0.00566704  0.05835889  0.09326369 -0.10082925
   0.02273228  0.12492627  0.01541135  0.01384074  0.04236639  0.03739384
  -0.03677399 -0.03102384  0.03256748  0.06240878 -0.05075281 -0.00656504
  -0.00130973  0.05777183 -0.05231229 -0.03974625  0.03634124]]


C:\ProgramData\Anaconda3\envs\env_3_6\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [37]:
clf = lr(penalty='l1', solver='liblinear').fit(rain_x_tr, rain_y_tr)
print(clf.score(rain_x_te, rain_y_te), clf.score(rain_x_te_true, rain_y_te_true), clf.score(rain_x_te_false, rain_y_te_false))
print('Coefficients:', clf.coef_)

0.8464570161753838 0.49842621736715426 0.9456137574510735
Coefficients: [[-7.39551671e-03  3.33702003e-03  8.02832419e-03 -2.79659177e-03
  -1.21436535e-01  6.03034025e-02 -1.80486156e-02 -2.97133953e-02
   1.82910441e-03  6.07634044e-02 -5.75195872e-03 -1.45968911e-02
   8.04195284e-04  1.21305131e-01  1.19444729e-02  3.25488965e-02
   5.44320698e-01 -7.25334561e-03 -2.92644924e-04 -3.70554952e-02
   4.01109362e-02  1.37734476e-01 -5.09364115e-02  1.77907591e-02
  -5.26665595e-02  1.72194201e-01 -8.69557410e-02 -9.59326960e-02
   1.25558284e-02]]


C:\ProgramData\Anaconda3\envs\env_3_6\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [39]:
U[:13].shape

(13, 5000)

In [42]:
rain_x_tr.shape

(97432, 29)

In [ ]:
# run pca for different numbers of output features, see how loss changes
# comparison with lasso (ridge)
# and RF to see loss progression